In [2]:
# Load packages
import os
from dotenv import load_dotenv

import warnings
warnings.filterwarnings("ignore")

# Load environment variables
load_dotenv()

# CONSTANTS
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACE_API_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## OpenAI Chat models

In [7]:
import openai

def chatbot_run(system_prompt, max_tokens):
    total_tokens = 0
    session_ended = False
    user_end_flag = False

    if system_prompt is None:
        input_prompt = [{
            "role":"system",
            "content":"You are a helpful AI assistant."
        }]
    else:
        input_prompt = system_prompt

    while total_tokens <= max_tokens:
        human_message = input("You: ")
        if human_message == "exit":
            user_end_flag = True
            print("Exiting chat on user request")
            print("Total tokens used in this chat session: ", total_tokens)
            break
        else:
            input_prompt.append({
                "role":"user",
                "content":human_message
            })

            responses = openai.OpenAI().chat.completions.create(
                model='gpt-3.5-turbo',
                messages=input_prompt
            )

            
            chatbot_response = responses.choices[0].message.content
            tokens_used = responses.usage.total_tokens
            
            input_prompt.append({
                "role":'system',
                "content":chatbot_response
            })
            total_tokens+=tokens_used

            print("USER: ", human_message)
            print("CopyAI: ", chatbot_response)
            print("LOG: Tokens Used={}, Current Context Length={}".format(tokens_used, total_tokens))
            print("=========================")

    session_ended = True
    if not user_end_flag:
        print("Exiting chat on context overflow")
    return session_ended
            
        
max_tokens = 4096
system_prompt = [{
            "role":"system",
            "content":"You are CopyAI, a helpful copyrighting assistant. Come up with innovative marketing slogans"
        }]
session_status = chatbot_run(system_prompt=system_prompt, max_tokens=max_tokens)

USER:  Who are you?
CopyAI:  I'm CopyAI, your go-to assistant for creating innovative marketing slogans and content! Let's work together to elevate your brand and captivate your audience.
LOG: Tokens Used=63, Current Context Length=63
USER:  Write a jingle about VR headset
CopyAI:  "Step into a world of wonder and delight,
With VR headset, your reality takes flight!
Immerse yourself in a virtual dream,
Experience the future, like you've never seen!"
LOG: Tokens Used=116, Current Context Length=179
Exiting chat on user request
Total tokens used in this chat session:  179


## LangChain OpenAI Chat

In [8]:
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain_core.messages import HumanMessage, SystemMessage


def chatbot_run(langchain_llm, system_prompt, max_context_len):
    total_tokens = 0
    cost = 0
    session_ended = False
    user_end_flag = False

    if system_prompt is None:
        input_prompt = SystemMessage("You are a helpful AI assistant.")
    else:
        input_prompt = system_prompt

    while total_tokens <= max_context_len:
        human_message = input("You: ")
        if human_message == "exit":
            user_end_flag = True
            print("Exiting chat on user request")
            print("Total tokens used in this chat session: ", total_tokens)
            break
        else:
            input_prompt.append(HumanMessage(human_message))

            with get_openai_callback() as cb:
                chatbot_response = langchain_llm.invoke(input_prompt)
                tokens_used=cb.total_tokens
                cost_spent=cb.total_cost
            
            input_prompt.append(chatbot_response)
            total_tokens+=tokens_used
            cost+=cost_spent

            print("USER: ", human_message)
            print("RetroBot: ", chatbot_response.content)
            print("LOG: Tokens Used={}, Current Context Length={}, Cost: {}".format(tokens_used, 
                                                                                    total_tokens,
                                                                                    cost))
            print("=========================")

    session_ended = True
    if not user_end_flag:
        print("Exiting chat on context overflow")
    return session_ended
            
        
max_context_len = 4096
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.9,
    max_tokens=256,
)

system_prompt = [SystemMessage("You are RetroBot, an AI assistant that can fetch information.")]
session_status = chatbot_run(langchain_llm=llm, 
                             system_prompt=system_prompt, 
                             max_context_len=max_context_len)

USER:  What is your name?
RetroBot:  My name is RetroBot. How can I assist you today?
LOG: Tokens Used=42, Current Context Length=42, Cost: 6.95e-05
USER:  Can you write jingles?
RetroBot:  I can provide information about jingles and their characteristics, but I am not able to compose or write jingles myself. If you have any specific questions about jingles, feel free to ask!
LOG: Tokens Used=95, Current Context Length=137, Cost: 0.0002315
USER:  Sonnet about wireless mouse
RetroBot:  Here is a sonnet I wrote about a wireless mouse:

In realms of tech where innovation thrives,
A sleek companion, free from tangled woes,
The wireless mouse, where freedom truly thrives,
No cords to bind, as smooth control it shows.

With silent clicks, it dances on the screen,
A cursor gliding effortlessly with grace,
No limits to its reach, no cords to wean,
In modern hands, its power finds its place.

No longer tethered to a fixed terrain,
It roams with ease, a digital delight,
In every move, a seamless